# Respostas Questionário II - SME0828

**Aluno:** Luiz Fernando Rabelo (11796893)

### Bibliotecas Utilizadas

Para a resolução do questionário, foram utilizadas as bibliotecas _numpy_, _pandas_, _matplotlib_, _requests_, _beautifulsoup_ e _io_ as quais são importadas abaixo:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup as bs
from io import StringIO

### Questão 01

In [ ]:
# Leitura do dataframe:
df = pd.read_csv('data/Life Expectancy Data.csv')

# Limpeza dos dados:
df.drop(df[np.isnan(df['Year'])].index, inplace=True)
df.drop(df[np.isnan(df['Life expectancy '])].index, inplace=True)

# Especificação dos países de interesse e filtro do dataframe:
nomes_paises = ['Ireland', 'Finland', 'Netherlands', 'Zambia', 'Angola']
df_filtrado = df[df['Country'].isin(nomes_paises)]

# Exibição do gráfico com a série temporal:
plt.figure(figsize=(10, 6))
for nome_pais in nomes_paises:
    anos = df_filtrado.loc[df_filtrado['Country'] == nome_pais, 'Year']
    expectativas = df_filtrado.loc[df_filtrado['Country'] == nome_pais, 'Life expectancy ']
    plt.plot(anos, expectativas, label=nome_pais)
plt.xlabel('Ano')
plt.ylabel('Expectativa de Vida')
plt.title('Série Temporal da Expectativa de Vida em Diferentes Países')
plt.legend()
plt.grid(True)
plt.show()

# Exibição das expectativas de vida em 2014:
display(df_filtrado[df_filtrado['Year'] == 2014][['Country', 'Life expectancy ']])

### Questão 02

In [ ]:
# Leitura do dataframe:
df = pd.read_csv('data/Life Expectancy Data.csv')

# Limpeza dos dados:
df.drop(df[np.isnan(df['Year'])].index, inplace=True)
df.drop(df[np.isnan(df['Life expectancy '])].index, inplace=True)

# Plot do histograma:
plt.figure(figsize=(10,6))
n, bins, patches = plt.hist(df['Life expectancy '], bins=10, density=True, width=5, edgecolor='black')
plt.title('Expectativa de Vida')
plt.xlabel('x (expectativa de vida)')
plt.ylabel('f(x)')
plt.xticks()
plt.yticks()
plt.show()

# Cálculo da média:
print(f'Média da Expectativa de Vida: {np.mean(np.array(df["Life expectancy "]))}')
print(f'Variância da Expectativa de Vida: {np.var(np.array(df["Life expectancy "]))}')

### Questão 03

In [ ]:
# Leitura do dataframe:
df = pd.read_csv('data/Life Expectancy Data.csv')

# Limpeza dos dados:
df.drop(df[np.isnan(df['Year'])].index, inplace=True)
df.drop(df[np.isnan(df['Life expectancy '])].index, inplace=True)

# Especificação dos países de interesse e filtro do dataframe:
nomes_paises = ['Ireland', 'Finland', 'Netherlands', 'Zambia', 'Angola']
df_filtrado = df[df['Country'].isin(nomes_paises)]

display(df_filtrado.groupby(['Country']).median(numeric_only=True)[['Schooling']])

O país com maior nível de escolaridade, em termos de mediana, é a Irlanda.

### Questão 04

In [ ]:
# Definição da url para a página:
url = 'https://www.worldometers.info/world-population/population-by-country'

# Requisição do conteúdo da página:
r = requests.get(url)

# Parse da página para estrutura do Beautiful Soup:
soup = bs(r.content)

# Extração dos nomes das colunas e do conteúdo dos dados:
colunas = [th.get_text() for th in soup.find('thead').find('tr').find_all('th')]
dados = [[td.get_text() for td in row.find_all('td')] for row in soup.find('tbody').find_all('tr')]

# Criação de dataframe com as informações extraídas:
df = pd.DataFrame(dados, columns=colunas, index=None)

# Remoção de vírgulas em dados numéricos:
colunas_virgula = [2, 4, 5, 6, 7]
for cv in colunas_virgula:
    df[colunas[cv]] = df[colunas[cv]].str.replace(',', '')

# Remoção de sinais de percentual em dados numéricos:
colunas_porcentagem = [3, 10, 11]
for cp in colunas_porcentagem:
    df[colunas[cp]] = df[colunas[cp]].str.replace(' %', '')
df.replace('N.A.', None, inplace=True)

# Conversão de valores de string para float:
for cs in range(2, df.shape[1]):
    df[colunas[cs]] = df[colunas[cs]].replace('', '0')
    df[colunas[cs]] = df[colunas[cs]].astype(float)

# Substituição de valores nulos por suas médias:
nulos = df.isnull().sum()
colunas_nulos = df[nulos[nulos > 0].index]
df.fillna(colunas_nulos.mean(), inplace=True)

# Construção da matriz de correlação:
matriz_correlacao = df.corr(numeric_only=True)

# Exibição gráfica da matriz:
plt.figure(figsize=(5,4))
plt.imshow(matriz_correlacao, cmap='gist_yarg', interpolation='none', aspect='auto')
plt.colorbar()
plt.xticks(range(len(matriz_correlacao)), matriz_correlacao.columns, rotation='vertical')
plt.yticks(range(len(matriz_correlacao)), matriz_correlacao.columns)
plt.suptitle('Correlação Entre as Variáveis')
plt.grid(False)
plt.show()

### Questão 05

In [ ]:
# Definição da url para a base de dados:
url = 'https://gist.githubusercontent.com/aishwarya8615/89d9f36fc014dea62487f7347864d16a/raw/8629d284e13976dcb13bb0b27043224b9266fffa/Life_Expectancy_Data.csv'

# Requisição do conteúdo:
resposta = requests.get(url)

# Criação de dataframe com o conteúdo textual obtido:
df = pd.read_csv(StringIO(resposta.text))

contagem_por_status = df['Status'].value_counts()

# Criação do gráfico de setores para a variável status:
plt.figure(figsize=(6,6))
plt.title('Distribuição de Países por Status')
plt.pie(contagem_por_status, labels=contagem_por_status.index, autopct='%1.1f%%', explode=[0, 0.05]);